In [255]:
import numpy as np
import scipy
import scipy.sparse as sparse
import matplotlib.pyplot as plt
import time
import imageio
import os
from copy import deepcopy

In [271]:
params = {
    "height":               400,
    "x_dom":                400,
    "y_dom":                  1,
    "t_max":                100,
    "r":                    0.3,
    "dt_snapshot":            1,          
    "n_ptcl_snapshot":   np.inf,
    "num_threads":            4,
    "all_resample":       False,
    "snapshot_resample":  False,
}

In [272]:
def get_nearest_neighbors(matrix, index, self = False):
    """
    Returns the indexes of the nearest neighbors to a given index in a 2D matrix.

    Args:
        matrix (numpy.ndarray): The 2D matrix.
        index (tuple): The given index (row, column).

    Returns:
        list: The indexes of the nearest neighbors.
    """
    # Get the number of rows and columns in the matrix
    rows, cols = matrix.shape

    # Unpack the given index
    row, col = index

    # Define the offsets for the neighbors (up, down, left, right)
    offsets = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    if self:
        offset = offset.append((0, 0))

    # List to store the valid neighbor indexes
    neighbor_indexes = []

    # Check each offset to determine the neighbor indexes
    for offset in offsets:
        new_row = row + offset[0]
        new_col = col + offset[1]

        # Check if the new index is within the matrix bounds
        if 0 <= new_row < rows:
            new_row
        else: new_row = -1
        if 0 <= new_col < cols:
            new_col
        else: new_col = -1
            
        neighbor_indexes.append((new_row, new_col))


    return neighbor_indexes

In [273]:
def plot_surface(surface, show = True, title = "Ballistic Deposition", 
                 colorbar = False, save = False, name = None):
    fig = plt.figure()
    plt.imshow(surface, cmap='binary', origin='lower')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(title)
    if colorbar:
        plt.colorbar()
    if save:
        if len(name) == 0:
           raise ValueError("No Name for surface plot")
        else: plt.savefig("SimResults/"+name)
        plt.close()
    # plt.show()
    if show:
        plt.show(fig)

In [274]:
def update_propensities(propensities, index, params, dt = 1):
    r = params["r"]
    propensities = np.power(r, dt)*propensities
    x_ind, y_ind = index

    propensities[x_ind, y_ind] = 1

    near_neighbors = get_nearest_neighbors(propensities, (x_ind, y_ind))
    for neighbor in near_neighbors:
        propensities[neighbor[0], neighbor[1]] = 1
    return propensities

def get_nonzero_probabilities(propensities):
    index_flat = list(zip(*np.nonzero(propensities))) #list of all indexes to use for choice

    x_ind, y_ind = np.nonzero(propensities)
    nonzero_propensities = propensities[x_ind, y_ind]
    probability = nonzero_propensities/np.sum(nonzero_propensities)

    return index_flat, probability

def choose_from_tau(taus):
    x_ind, y_ind = np.unravel_index(np.argmin(taus), taus.shape)
    tau = taus[x_ind, y_ind]
    return (x_ind, y_ind), tau

In [275]:
def add_point(index, space, max_height):
    x_ind, y_ind = index
    near_neighbors = get_nearest_neighbors(max_height, (x_ind, y_ind))
    height_surrondings = [max_height[neighbor[0], neighbor[1]] for neighbor in near_neighbors]

    highest_pos = max([max(height_surrondings), max_height[x_ind, y_ind]+1])
    space[x_ind, y_ind, highest_pos] = 1
    max_height[x_ind, y_ind] = highest_pos
    return space, max_height

In [276]:
height = params["height"]
width = params["x_dom"]
length = params["y_dom"]
r = params["r"]
t = n_ptcls = n_snapshot = 0

space = np.zeros((width, length, height), dtype=int) #actual simulation space
max_height = np.zeros((width, length), dtype=int) #occupation/height at each site
propensities = np.ones((width, length), dtype = float) #probability of droping at each coordinate
index_flat, _ = get_nonzero_probabilities(propensities)
taus = np.random.exponential(1/propensities)

In [277]:
while(t<params["t_max"]):
    try:
        index_chosen, tau_min = choose_from_tau(taus)
        t += tau_min
        space, max_height = add_point(index_chosen, space, max_height)
        propensities_new = update_propensities(propensities, index_chosen, params, dt = tau_min)

        for tau_row, row2, row3 in zip(taus, propensities_new, propensities):
            for i, p_new, p in zip(range(len(tau_row)), row2, row3):
                if tau_row[i] == tau_min or params["all_resample"]:
                    tau_row[i] = np.random.exponential(1/p_new)
                elif p_new > 0 :
                    tau_row[i] = (p/p_new)*(tau_row[i] - tau_min)
                else:
                    tau_row[i] = np.inf

        propensities = propensities_new

        if (
            t > n_snapshot*params["dt_snapshot"]
            ) or (
                n_ptcls%params["n_ptcl_snapshot"] == 0
                ):
            plot_surface(space[:, 0, :].transpose(),title=f"Ballistic Deposition time: {t}",  
                         save= True, name = f"frame_{n_snapshot}.png")
            n_snapshot += 1
        n_ptcls += 1

    except IndexError:
        print(f"Fully Occupied at time: {t}| N_Ptcls: {n_ptcls}| N_snapshots: {n_snapshot}")
        break

    except KeyboardInterrupt:
        print(f"Manually Stopped at time: {t}| N_Ptcls: {n_ptcls}| N_snapshots: {n_snapshot}")
        break
else:
    print(f"Stopped at time: {t}| N_Ptcls: {n_ptcls}| N_snapshots: {n_snapshot}")

Stopped at time: 100.08780142247257| N_Ptcls: 4305| N_snapshots: 101


In [278]:
foldername = "SimResults"
frames = []
n_updates = 0

while(True):
    try:
        image = imageio.v2.imread(f'./{foldername}/frame_{n_updates}.png')
        frames.append(image)
        n_updates += 1
    except FileNotFoundError: 
        break

imageio.mimsave(f'./AniTest_r_{params["r"]}.gif', 
                frames, fps = 15)